In [1]:
from time import time

import torch
import sklearn.datasets
import sklearn.preprocessing
import sklearn.model_selection
import numpy as np
import math
import matplotlib.pyplot as plt

from torchvision.datasets import MNIST
from torchvision.datasets import FashionMNIST as FMNIST
from torchvision.datasets import EMNIST
import torchvision.transforms as transforms

import Onlinehd

SEED = 1234
device = 'cuda' if torch.cuda.is_available() else 'cpu'
np.random.seed(SEED)
torch.manual_seed(SEED)

In [2]:
def load():
    download_root = 'data/emnist_dataset'
    
    temp = EMNIST(download_root, split = 'letters', train=True, download=True)
    x = temp.data.unsqueeze(3).numpy().transpose((0,2,1,3))
    y = temp.targets.numpy() - 1
    temp = EMNIST(download_root, split='letters', train=False, download=True)
    x_test = temp.data.unsqueeze(3).numpy().transpose((0,2,1,3))
    y_test = temp.targets.numpy() - 1
    
    x = torch.from_numpy(x).float()
    y = torch.from_numpy(y).long()
    x_test = torch.from_numpy(x_test).float()
    y_test = torch.from_numpy(y_test).long()
    
    x_test = x_test.float()
    y_test = y_test.long().squeeze()

    if len(x.shape) != 3:
        x = x.squeeze(3)
        x_test = x_test.squeeze(3)
    
    return x, x_test, y, y_test

In [4]:
train_x, x_test, train_y, y_test = load()

/Users/jeongseongheon/opt/anaconda3/envs/UCI/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
train_x = train_x.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

f_scaler = sklearn.preprocessing.Normalizer().fit(train_x.reshape(-1, 784))
train_x = torch.tensor(f_scaler.transform(train_x.reshape(-1, 784)))
x_test = torch.tensor(f_scaler.transform(x_test.reshape(-1, 784)))

In [6]:
from time import time

f_classes = train_y.unique().size(0)
f_features = train_x.size(1)
model = Onlinehd.OnlineHD(f_classes, f_features, dim=10000)

if torch.cuda.is_available():
    f_x = f_x.cuda()
    f_y = f_y.cuda()
    f_x_test = f_x_test.cuda()
    f_y_test = f_y_test.cuda()
    Fmodel = Fmodel.to('cuda')
    print('Using GPU!')

print('Training...')
t = time()
model = model.fit(train_x, train_y, bootstrap=1.0, lr=0.035, epochs=20)
t = time() - t

Training...
0
10


In [7]:
print('Validating...')
f_yhat = model(train_x)
f_yhat_test = model(x_test)

f_acc = (train_y == f_yhat).float().mean()
f_acc_test = (y_test == f_yhat_test).float().mean()

print(f'{f_acc = :6f}')
print(f'{f_acc_test = :6f}')
print(f'{t = :6f}')

Validating...
f_acc = 0.860625
f_acc_test = 0.832452
t = 87.080077


In [10]:
def pgd_attack(x_original, y_target, model, alpha, iterations=10, epsilon=0.03):
    criterion = nn.CosineEmbeddingLoss()
    x_perturbed = x_original.clone().detach()
    x_perturbed.requires_grad = True
    
    for _ in range(iterations):
        output = model.encode(x_perturbed)
        label = torch.ones(26, dtype=int) * -1
        label[y_target.item()] = 1
        loss = criterion(output, model.model, label)
                
#         loss.backward()
#         grad = x_perturbed.gard.data
        grad = torch.autograd.grad(loss, x_perturbed)[0]
        x_perturbed = x_perturbed.detach() + alpha*grad.sign()
        eta = torch.clamp(x_perturbed - x_original, min=-epsilon, max=epsilon)
        x_perturbed = x_original + eta
    return x_perturbed

In [11]:
import time
import torch.nn as nn

epsilons = torch.Tensor([0.01, 0.02, 0.03, 0.04, 0.05])
acc_results = dict()
verbose = False
N_VAL_SAMPLES = x_test.shape[0]
criterion = nn.CosineEmbeddingLoss()

PGD_test_attack_data = []
PGD_test_attack_label = []

for eps in epsilons:
    correct_unperturbed = 0
    correct_perturbed = 0
    t0 = time.perf_counter()
    
    for j in range(len(x_test)):
        if j % 1000 == 0 :
            print(j)
#     for j, val_data in enumerate(x, 0):
        ### NOTE: IT WOULD BE MORE EFFICIENT TO ITERATE ONLY ONCE THROUGH THE DATA AND PERFORM ALL THE ATTACKS
        x, y_target = x_test[j].reshape(-1, 784), y_test[j]
        x, y_target = x.to(device), y_target.to(device)
        x.requires_grad = True
        
        y_pred = model(x)
        
        if y_pred == y_target: # Only make attack on correctly classified samples
            correct_unperturbed += 1
            # Calculate loss and gradient
            perturbed_x = pgd_attack(x, y_target, model=model, alpha=eps)
            PGD_test_attack_data.append(perturbed_x)
            PGD_test_attack_label.append(y_target)
            
            y_pred_perturbed = model(perturbed_x)
#             y_pred_perturbed = torch.argmin(perturbed_output)
#             loss_perturbed = criterion(perturbed_output, model.model, label)
            if y_pred_perturbed == y_target:
                correct_perturbed += 1
                
    acc_before_attack = correct_unperturbed / N_VAL_SAMPLES
    acc_after_attack = correct_perturbed / N_VAL_SAMPLES
    print(f'\nPGD Attack with alpha = {eps:.5f} | Elapsed time: {time.perf_counter() - t0:.2f} seconds.')
    print(f'Accuracy: Before the attack -> {100 * acc_before_attack:.2f}%\t|\tAfter the attack -> {100 * acc_after_attack:.2f}%')
    acc_results[eps.item()] = acc_after_attack
acc_results[0] = acc_before_attack

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000

PGD Attack with alpha = 0.01000 | Elapsed time: 889.27 seconds.
Accuracy: Before the attack -> 83.24%	|	After the attack -> 31.96%
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000

PGD Attack with alpha = 0.02000 | Elapsed time: 895.12 seconds.
Accuracy: Before the attack -> 83.24%	|	After the attack -> 34.27%
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000

PGD Attack with alpha = 0.03000 | Elapsed time: 891.71 seconds.
Accuracy: Before the attack -> 83.24%	|	After the attack -> 39.90%
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000

PGD Attack with alpha = 0.04000 | Elapsed time: 891.27 seconds.
Accuracy: Before the attack -> 83.24%	|	After the attack -> 38.61%
0
1000
2000
3000
400

In [12]:
PGD_test_attack_label
for i in range(len(PGD_test_attack_label)):
    PGD_test_attack_label[i] = PGD_test_attack_label[i].item()

In [13]:
PGD_test_attack_label = torch.tensor(PGD_test_attack_label).long()

In [14]:
len(PGD_test_attack_data)

86570

In [15]:
PGD_test_attack = torch.zeros(86570, 784)

for i in range(len(PGD_test_attack_data)):
    PGD_test_attack[i] = PGD_test_attack_data[i].reshape(784)

In [16]:
train_data = f_scaler.transform(PGD_test_attack.detach().numpy())

In [17]:
train_data = torch.from_numpy(train_data).float()

# Adversarial Training

In [18]:
from time import time

classes = train_y.unique().size(0)
features = train_x.size(1)
model = Onlinehd.OnlineHD(classes, features, dim=10000)

if torch.cuda.is_available():
    train_x = train_x.cuda()
    train_y = train_y.cuda()
    x_test = x_test.cuda()
    y_test = y_test.cuda()
    model = model.to('cuda')
    print('Using GPU!')

print('Training...')
t = time()
model = model.fit(train_data, PGD_test_attack_label, bootstrap=1.0, lr=0.035, epochs=20)
t = time() - t

Training...
0
10


In [19]:
print('Validating...')
yhat = model(train_data)
yhat_test = model(x_test)
acc = (PGD_test_attack_label == yhat).float().mean()
acc_test = (y_test == yhat_test).float().mean()
print(f'{acc = :6f}')
print(f'{acc_test = :6f}')
print(f'{t = :6f}')

Validating...
acc = 0.881160
acc_test = 0.646394
t = 46.596119


In [20]:
import pickle

with open('hd_adversarial_sample/EMNIST_HD_FGSM.pickle', 'rb') as f:
    FGSM = pickle.load(f)

In [21]:
FGSM_001 = FGSM['data']['0.01']
FGSM_003 = FGSM['data']['0.03']
FGSM_007 = FGSM['data']['0.07']
FGSM_01 = FGSM['data']['0.1']

In [22]:
import pickle

with open('hd_adversarial_sample/EMNIST_HD_DF.pickle', 'rb') as f:
    DF = pickle.load(f)

In [23]:
DF_data = DF['data']

In [24]:
import pickle

with open('hd_adversarial_sample/EMNIST_HD_JSMA.pickle', 'rb') as f:
    JSMA = pickle.load(f)

In [25]:
JSMA_data = JSMA['data']

In [29]:
FGSM_001 = torch.from_numpy(f_scaler.transform(FGSM_001.detach().numpy().reshape(-1, 784))).float()
FGSM_003 = torch.from_numpy(f_scaler.transform(FGSM_003.detach().numpy().reshape(-1, 784))).float()
FGSM_007 = torch.from_numpy(f_scaler.transform(FGSM_007.detach().numpy().reshape(-1, 784))).float()
FGSM_01 = torch.from_numpy(f_scaler.transform(FGSM_01.detach().numpy().reshape(-1, 784))).float()
DF_data = torch.from_numpy(f_scaler.transform(DF_data.reshape(-1, 784))).float()
JSMA_data = torch.from_numpy(f_scaler.transform(JSMA_data.detach().numpy().reshape(-1, 784))).float()

In [30]:
print('Validating...')

FGSM_001_yhat = model(FGSM_001)
FGSM_003_yhat = model(FGSM_003)
FGSM_007_yhat = model(FGSM_007)
FGSM_01_yhat = model(FGSM_01)
DF_data_yhat = model(DF_data)
JSMA_data_yhat = model(JSMA_data)

FGSM_001_acc = (y_test == FGSM_001_yhat).float().mean()
FGSM_003_acc = (y_test == FGSM_003_yhat).float().mean()
FGSM_007_acc = (y_test == FGSM_007_yhat).float().mean()
FGSM_01_acc = (y_test == FGSM_01_yhat).float().mean()
DF_acc = (y_test == DF_data_yhat).float().mean()
JSMA_acc = (y_test == JSMA_data_yhat).float().mean()

print(f'{FGSM_001_acc = :6f}')
print(f'{FGSM_003_acc = :6f}')
print(f'{FGSM_007_acc = :6f}')
print(f'{FGSM_01_acc = :6f}')
print(f'{DF_acc = :6f}')
print(f'{JSMA_acc = :6f}')

Validating...
FGSM_001_acc = 0.667933
FGSM_003_acc = 0.687308
FGSM_007_acc = 0.636683
FGSM_01_acc = 0.564087
DF_acc = 0.656490
JSMA_acc = 0.613510
